<p style="text-align:center">
    <a href="https://www.ptit.edu.vn" target="_blank">
        <img src="https://i.imgur.com/XBSxCH1.png" width="400" alt="PTIT Logo">
    </a>
</p>


<center>

# **Practice lab: Cardiovascular Prediction**

</center>


### THƯ VIỆN 

In [36]:
import pandas as pd
import numpy as np

import copy

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from sklearn import preprocessing
np.random.seed(2095)

### ĐỌC DỮ LIỆU TỪ DATASET

In [37]:
file_path = r'D:\PTIT-CNN-Prediction-Cardiovascular\CardiacPrediction.xlsx'
df = pd.read_excel(file_path, sheet_name='CoroHeartDis')

### KHAI PHÁ DỮ LIỆU

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37079 entries, 0 to 37078
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SEQN                         37079 non-null  int64  
 1   Gender                       37079 non-null  int64  
 2   Age                          37079 non-null  int64  
 3   Annual-Family-Income         37079 non-null  int64  
 4   Ratio-Family-Income-Poverty  37079 non-null  float64
 5   X60-sec-pulse                37079 non-null  int64  
 6   Systolic                     37079 non-null  int64  
 7   Diastolic                    37079 non-null  int64  
 8   Weight                       37079 non-null  float64
 9   Height                       37079 non-null  float64
 10  Body-Mass-Index              37079 non-null  float64
 11  White-Blood-Cells            37079 non-null  float64
 12  Lymphocyte                   37079 non-null  float64
 13  Monocyte        

In [39]:
df.describe()

,SEQN,Gender,Age,Annual-Family-Income,Ratio-Family-Income-Poverty,X60-sec-pulse,Systolic,Diastolic,Weight,Height,...,Total-Cholesterol,HDL,Glycohemoglobin,Vigorous-work,Moderate-work,Health-Insurance,Diabetes,Blood-Rel-Diabetes,Blood-Rel-Stroke,CoronaryHeartDisease
count,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,...,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000
mean,48901.041236,1.513282,48.943661,7.358208,2.559026,72.579250,124.090078,69.919253,80.988276,167.389601,...,5.081713,1.370344,5.676496,1.783840,1.598856,1.218587,1.907333,1.549502,1.796165,0.040670
std,26753.636441,0.499830,18.010440,3.994083,1.624789,12.242108,19.254741,13.575804,20.678734,10.122908,...,1.072682,0.415985,1.050223,0.448324,0.511199,0.461102,0.349674,0.497550,0.402853,0.197527
min,2.000000,1.000000,20.000000,1.000000,0.000000,32.000000,0.000000,0.000000,32.300000,129.700000,...,1.530000,0.160000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,26120.500000,1.000000,33.000000,4.000000,1.140000,64.000000,111.000000,62.000000,66.500000,160.000000,...,4.320000,1.070000,5.200000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,0.000000
50%,50065.000000,2.000000,48.000000,7.000000,2.180000,72.000000,121.000000,70.000000,78.200000,167.100000,...,5.020000,1.290000,5.400000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
75%,71173.500000,2.000000,63.000000,10.000000,4.130000,80.000000,134.000000,78.000000,92.100000,174.600000,...,5.740000,1.600000,5.800000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
max,93702.000000,2.000000,85.000000,15.000000,5.000000,224.000000,270.000000,132.000000,371.000000,204.500000,...,14.090000,5.840000,18.800000,3.000000,3.000000,9.000000,3.000000,2.000000,2.000000,1.000000


### GIẢI THÍCH BỘ DỮ LIỆU LÂM SÀNG
- Chỉ số nhân khẩu học
    - Gender, Age: Giới tính và tuổi của bệnh nhân
    - SEQN: Mã định danh người tham gia
    - Annual-Family-Income: Thu nhập hàng năm của hộ gia đình
    - Ratio-Family-Income-Poverty: Tỷ lệ thu nhập hộ gia đình và ngưỡng nghèo liên bang
- Thông tin sinh lý cơ bản
    - X60-sec-pulse: Nhịp tim (số lần đập/phút)
    - Systolic: Huyết áp tâm thu
    - Diastolic: Huyết áp tâm trương
    - Weight: Trọng lượng
    - Height: Chiều cao
    - Body-Mass-Index: Chỉ số khối cơ thể (BMI)
- Chỉ số huyết học
    - White-Blood-Cells (WBC): Số lượng bạch cầu (10^3/μL), giúp đánh giá tình trạng nhiễm trùng hoặc các bệnh lý miễn dịch
    - Lymphocyte: Phần trăm lymphocyte, một loại bạch cầu liên quan đến miễn dịch
    - Monocyte: Phần trăm monocyte, tham gia vào việc chống lại nhiễm trùng
    - Eosinophils: Phần trăm bạch cầu ái toan, thường tăng trong các phản ứng dị ứng hoặc nhiễm ký sinh trùng
    - Basophils: Phần trăm bạch cầu ưa kiềm, liên quan đến các phản ứng dị ứng
    - Red-Blood-Cells (RBC): Số lượng hồng cầu (10^6/μL), đánh giá khả năng vận chuyển oxy
    - Hemoglobin (Hb): Nồng độ hemoglobin (g/dL), chỉ số quan trọng trong đánh giá thiếu máu
    - Mean-Cell-Vol (MCV): Thể tích trung bình của hồng cầu, giúp xác định loại thiếu máu
    - Mean-Cell-Hgb-Conc. (MCHC): Nồng độ hemoglobin trung bình trong một hồng cầu
    - Mean-cell-Hemoglobin (MCH): Lượng hemoglobin trung bình trong một hồng cầu
    - Platelet-count: Số lượng tiểu cầu (10^3/μL), liên quan đến khả năng đông máu
    - Mean-Platelet-Vol: Thể tích trung bình của tiểu cầu
    - Segmented-Neutrophils: Phần trăm bạch cầu trung tính, quan trọng trong đáp ứng miễn dịch
    - Hematocrit: Tỷ lệ phần trăm tế bào máu trong tổng thể tích máu
    - Red-Cell-Distribution-Width (RDW): Độ phân phối kích thước hồng cầu, thường tăng trong các dạng thiếu máu
- Chỉ số sinh hóa máu
    - Albumin: Protein trong huyết thanh, phản ánh tình trạng dinh dưỡng và chức năng gan
    - ALP (Alkaline Phosphatase): Enzym phosphatase kiềm, liên quan đến chức năng gan, mật và xương
    - AST (Aspartate Transaminase): Enzym liên quan đến tổn thương tế bào gan hoặc cơ tim
    - ALT (Alanine Transaminase): Enzym chỉ điểm tổn thương gan
    - Cholesterol: Cholesterol toàn phần (mmol/L), liên quan đến nguy cơ bệnh tim mạch
    - Creatinine: Sản phẩm thoái hóa của cơ, chỉ số chức năng thận
    - Glucose: Nồng độ đường trong máu, dùng để chẩn đoán bệnh tiểu đường
    - GGT (Gamma-Glutamyl Transferase): Enzym phản ánh tổn thương gan hoặc sử dụng rượu kéo dài
    - Iron: Hàm lượng sắt trong máu, liên quan đến bệnh thiếu máu hoặc quá tải sắt
    - LDH (Lactate Dehydrogenase): Enzym liên quan đến tổn thương tế bào
    - Phosphorus: Nồng độ photphat máu, quan trọng trong chức năng xương và thận
    - Bilirubin: Sản phẩm phân hủy của hemoglobin, chỉ điểm chức năng gan và mật
    - Protein: Tổng lượng protein trong huyết thanh
    - Uric.Acid: Nồng độ acid uric, liên quan đến bệnh gout
    - Triglycerides: Mỡ trung tính trong máu, liên quan đến bệnh lý tim mạch
    - Total-Cholesterol: Cholesterol toàn phần (mmol/L)
    - HDL: Cholesterol tốt (HDL), bảo vệ tim mạch
    - Glycohemoglobin (HbA1c): Chỉ số kiểm soát đường máu dài hạn, quan trọng trong bệnh tiểu đường
- Lối sống và yếu tố gia đình
    - Vigorous-work: Mức độ tham gia công việc nặng (mã hóa: 1 - Thấp, 3 - Cao)
    - Moderate-work: Mức độ tham gia công việc vừa (mã hóa tương tự)
    - Health-Insurance: Có bảo hiểm y tế hay không (1: Có, 2: Không)
    - Diabetes: Tiền sử bệnh tiểu đường (1: Có, 2: Không)
    - Blood-Rel-Diabetes: Có người thân bị tiểu đường (1: Có, 2: Không)
    - Blood-Rel-Stroke: Có người thân bị đột quỵ (1: Có, 2: Không)
- Biến mục tiêu
    - CoronaryHeartDisease: Biến mục tiêu, biểu thị bệnh tim mạch vành (0: Không mắc, 1: Mắc)

In [40]:
df['CoronaryHeartDisease'].value_counts()

CoronaryHeartDisease
0    35571
1     1508
Name: count, dtype: int64

📶  Có thể thấy sự mất cân bằng dữ liệu rất lớn trên tập dữ liệu này với tỉ lệ 1:35 (Người mắc bệnh : Người không mắc bệnh)

### TIỀN XỬ LÝ DỮ LIỆU

Kiểm tra các giá trị khuyết (Null, NaN)

In [41]:
null_counts = df.isnull().sum()
print(null_counts)

SEQN                           0
Gender                         0
Age                            0
Annual-Family-Income           0
Ratio-Family-Income-Poverty    0
X60-sec-pulse                  0
Systolic                       0
Diastolic                      0
Weight                         0
Height                         0
Body-Mass-Index                0
White-Blood-Cells              0
Lymphocyte                     0
Monocyte                       0
Eosinophils                    0
Basophils                      0
Red-Blood-Cells                0
Hemoglobin                     0
Mean-Cell-Vol                  0
Mean-Cell-Hgb-Conc.            0
Mean-cell-Hemoglobin           0
Platelet-count                 0
Mean-Platelet-Vol              0
Segmented-Neutrophils          0
Hematocrit                     0
Red-Cell-Distribution-Width    0
Albumin                        0
ALP                            0
AST                            0
ALT                            0
Cholestero

❌ Không có giá trị NULL trong bộ dữ liệu 

Kiểm tra trùng lặp

In [42]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Number of duplicate rows: 0


❌ Không có giá trị trùng lặp trong bộ dữ liệu 

### CHỌN LỌC BIẾN BẰNG HỒI QUY LASSO VÀ BỎ PHIẾU ĐA SỐ

### Hồi quy Lasso là gì?

**Hồi quy Lasso (Lasso Regression)** là một kỹ thuật trong phân tích dữ liệu và học máy, được sử dụng để thực hiện **hồi quy tuyến tính** trong đó có một bước điều chỉnh đặc biệt để giúp giảm thiểu mô hình quá phức tạp hoặc bị **overfitting**.

### Các Khái Niệm Cơ Bản:
- **Hồi quy tuyến tính (Linear Regression):** Là một mô hình thống kê dùng để dự đoán giá trị của biến phụ thuộc (y) từ các biến độc lập (x).
- **Overfitting:** Là hiện tượng khi mô hình học quá kỹ vào dữ liệu huấn luyện, dẫn đến việc không tổng quát được khi gặp dữ liệu mới.

### Lasso Regression:
- **Lasso** là viết tắt của **Least Absolute Shrinkage and Selection Operator**. Kỹ thuật này không chỉ thực hiện hồi quy tuyến tính mà còn có khả năng **chọn lọc đặc trưng** (feature selection).
- Lasso thêm vào một điều kiện ràng buộc (penalty term) vào trong hàm mất mát (loss function) của hồi quy tuyến tính.

$$
\text{Loss function} = \text{RSS} + \lambda \sum_{i=1}^{p} |\beta_i|
$$

Trong đó:
- **RSS (Residual Sum of Squares):** Tổng bình phương sai số dự đoán.
- **\(\beta_i\):** Các hệ số hồi quy của các biến độc lập.
- **\(\lambda\):** Hệ số điều chỉnh (regularization parameter), điều chỉnh mức độ ảnh hưởng của phần ràng buộc. Khi \(\lambda\) tăng, mức độ “co lại” của các hệ số hồi quy càng mạnh.

## Đặc Điểm Của Lasso:
1. **Feature Selection:** Lasso có thể đẩy một số hệ số hồi quy về 0, giúp loại bỏ các biến không có ảnh hưởng lớn. Điều này giúp chọn ra những đặc trưng quan trọng trong mô hình, giảm bớt độ phức tạp của mô hình.
2. **Điều chỉnh độ phức tạp:** Thông qua hệ số \(\lambda\), Lasso giúp điều chỉnh mô hình sao cho không bị overfitting, đồng thời vẫn duy trì khả năng dự đoán chính xác.
3. **Sử dụng trong dữ liệu có nhiều đặc trưng (features):** Lasso rất hữu ích khi bạn có một lượng lớn đặc trưng và muốn giảm số lượng đặc trưng đầu vào, đồng thời duy trì độ chính xác của mô hình.

### So Với Ridge Regression:
- Cả **Lasso** và **Ridge Regression** đều là các phương pháp hồi quy có điều chỉnh (regularized regression), nhưng khác biệt chính giữa chúng là:
  - **Lasso** sử dụng điều chỉnh L1 (làm giảm các hệ số về 0), trong khi **Ridge** sử dụng điều chỉnh L2 (làm giảm các hệ số về giá trị nhỏ, nhưng không phải bằng 0).
  - Lasso có khả năng loại bỏ các đặc trưng không quan trọng, trong khi Ridge thường chỉ làm giảm chúng mà không loại bỏ hoàn toàn.

### Ứng Dụng:
- Lasso regression thường được sử dụng khi bạn muốn:
  - Tìm ra các biến độc lập quan trọng nhất trong một mô hình.
  - Giảm thiểu overfitting trong các mô hình có nhiều biến.
  - Dự đoán trong các bài toán hồi quy với nhiều đặc trưng.

Nhờ vào khả năng chọn lọc đặc trưng và giảm độ phức tạp mô hình, Lasso là một công cụ mạnh mẽ trong phân tích và học máy.


### Tạo biến mục tiêu và loại bỏ các cột không cần thiết

In [43]:
opLabel = np.array(df['CoronaryHeartDisease'])

# Loại bỏ các cột không cần thiết
df.drop(
    ['SEQN', 'CoronaryHeartDisease', 'Annual-Family-Income', 'Height', 
     'Ratio-Family-Income-Poverty', 'X60-sec-pulse', 'Health-Insurance', 
     'Lymphocyte', 'Monocyte', 'Eosinophils', 'Total-Cholesterol', 
     'Mean-Cell-Vol', 'Mean-Cell-Hgb-Conc.', 'Hematocrit', 'Segmented-Neutrophils'], 
    axis=1, 
    inplace=True
)

In [44]:
# Chuyển đổi các biến phân loại thành biến giả
df = pd.get_dummies(df, columns=["Gender", "Diabetes", "Blood-Rel-Diabetes", 
                                 "Blood-Rel-Stroke", "Vigorous-work", "Moderate-work"])

# Kiểm tra số lượng biến sau chuyển đổi
print(f"Số lượng biến sau khi tạo biến giả: {df.shape[1]}")

# Lấy tên các cột biến đặc trưng và chuyển dataframe thành mảng numpy
varb = np.array(df.columns)
ipData = np.array(df)

# Khởi tạo mảng đếm tần suất (bỏ phiếu) cho các biến đặc trưng
featureVote = np.zeros(ipData.shape[1])

Số lượng biến sau khi tạo biến giả: 45


In [45]:
iteR = 100  # Số lần lặp lại

# Vòng lặp chọn lọc biến
for num in range(iteR):
    # Phân chia dữ liệu theo hai lớp
    label0_indx = np.where(opLabel == 0)[0]
    label1_indx = np.where(opLabel == 1)[0]
    numTrainData0 = 1300
    numTrainData1 = 1300

    # Xáo trộn chỉ số
    np.random.shuffle(label0_indx)
    np.random.shuffle(label1_indx)

    # Chia tập dữ liệu
    label0_indx_train = label0_indx[:numTrainData0-1]
    label1_indx_train = label1_indx[:numTrainData1-1]
    trainIndx = np.append(label0_indx_train, label1_indx_train)
    trainData = ipData[trainIndx]
    trainLabel = opLabel[trainIndx]

    # Chuẩn hóa dữ liệu
    scaler = preprocessing.StandardScaler().fit(trainData)
    trainData_scaled = scaler.transform(trainData)

    # Hồi quy Lasso
    regr = Lasso(random_state=0, alpha=0.006, tol=0.000001, max_iter=100000)
    regr.fit(trainData_scaled, trainLabel)

    # Ghi nhận các biến có trọng số khác 0
    cof = np.abs(regr.coef_)
    colIndx = np.where(cof != 0)[0]
    for col in colIndx:
        featureVote[col] += 1

In [46]:
print(featureVote)
print(iter)

[100.  33.  82.  39.  14.  91.  50.  67.   9.  19. 100.  49. 100.  47.
  26.  24.   9. 100.  88.  14.  36.  33. 100.  24.  47.  25. 100.  99.
  96.  95. 100.  59.  86.  73.   5.  87.  49. 100.  60.  14.  13.  89.
   2.  99.  64.]
<built-in function iter>


In [47]:
# Đặt ngưỡng tần suất
thresH = iteR // 5

# Chọn các biến có tần suất >= ngưỡng
featureIndx = np.where(featureVote[0:30] >= thresH)[0]

# Giữ lại các biến từ vị trí 30 trở đi
featureIndx = np.append(featureIndx, np.arange(30, ipData.shape[1]))

# Kiểm tra số lượng biến được chọn
print(f"Số lượng biến được chọn: {len(featureIndx)}")

# Lấy tên các biến được chọn
selected_features_via_threshold = varb[featureIndx]

# In các biến được chọn
print("Các biến đặc trưng được chọn:")
for feature in selected_features_via_threshold:
    print(feature)

Số lượng biến được chọn: 40
Các biến đặc trưng được chọn:
Age
Systolic
Diastolic
Weight
White-Blood-Cells
Basophils
Red-Blood-Cells
Platelet-count
Mean-Platelet-Vol
Red-Cell-Distribution-Width
Albumin
ALP
AST
Cholesterol
Creatinine
GGT
Iron
LDH
Phosphorus
Bilirubin
Protein
Uric.Acid
Triglycerides
HDL
Glycohemoglobin
Gender_1
Gender_2
Diabetes_1
Diabetes_2
Diabetes_3
Blood-Rel-Diabetes_1
Blood-Rel-Diabetes_2
Blood-Rel-Stroke_1
Blood-Rel-Stroke_2
Vigorous-work_1
Vigorous-work_2
Vigorous-work_3
Moderate-work_1
Moderate-work_2
Moderate-work_3


In [48]:
# Tìm các chỉ số của tất cả các biến trong tập dữ liệu gốc
tInx = np.arange(ipData.shape[1])

# Tìm các chỉ số của các biến không nằm trong danh sách được chọn
rrInx = tInx[~np.isin(tInx, featureIndx)]

# Lấy tên các biến bị loại bỏ
removed_features = varb[rrInx]
print(f"Số lượng biến bị loại bỏ: {len(removed_features)}")

# In danh sách các biến bị loại bỏ
print("Các biến đặc trưng bị loại bỏ sau chọn lọc:")
for feature in removed_features:
    print(feature)

Số lượng biến bị loại bỏ: 5
Các biến đặc trưng bị loại bỏ sau chọn lọc:
Body-Mass-Index
Hemoglobin
Mean-cell-Hemoglobin
ALT
Glucose


### GIẢM TẬP DỮ LIỆU VÀ CHIA TẬP DỮ LIỆU HUẤN LUYỆN & KIỂM TRA

**MỤC ĐÍCH**
- Giảm kích thước dữ liệu bằng cách chỉ giữ lại các biến đặc trưng đã chọn thông qua hồi quy Lasso
- Chia dữ liệu thành tập huấn luyện (train) và kiểm tra (test) với số lượng mẫu được xác định riêng cho từng lớp (lớp 0 và lớp 1)
- Chuẩn hóa dữ liệu để sẵn sàng sử dụng trong các mô hình học máy
- Cho phép thay đổi kích thước tập huấn luyện bằng cách chỉnh sửa các biến numTrainData0 (số mẫu của lớp 0) và numTrainData1 (số mẫu của lớp 1)

In [51]:
# Giảm tập dữ liệu với các biến đặc trưng đã chọn từ dataframe df
reduced_data = df.iloc[:, featureIndx].values

# Tìm chỉ số của hai lớp (0 và 1)
label0_indx = np.where(opLabel == 0)[0]  # Lớp 0 (không bị bệnh)
label1_indx = np.where(opLabel == 1)[0]  # Lớp 1 (bị bệnh)

# Định nghĩa số lượng mẫu cho tập huấn luyện
numTrainData0 = 4300  # Số mẫu lớp 0 trong tập huấn luyện
numTrainData1 = 1169  # Số mẫu lớp 1 trong tập huấn luyện

# Xáo trộn chỉ số
np.random.shuffle(label0_indx)
np.random.shuffle(label1_indx)

# Chia tập huấn luyện và kiểm tra
label0_indx_train = label0_indx[:numTrainData0]
label1_indx_train = label1_indx[:numTrainData1]
label0_indx_test = label0_indx[numTrainData0:]
label1_indx_test = label1_indx[numTrainData1:]

# Gộp chỉ số của hai lớp
testIndx = np.append(label0_indx_test, label1_indx_test)
trainIndx = np.append(label0_indx_train, label1_indx_train)

# Tạo tập huấn luyện và kiểm tra
x_train = reduced_data[trainIndx]
y_train = opLabel[trainIndx]
x_test = reduced_data[testIndx]
y_test = opLabel[testIndx]

# Chuẩn hóa dữ liệu
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Sao lưu dữ liệu chuẩn hóa
x_train_org = copy.deepcopy(x_train)
x_test_org = copy.deepcopy(x_test)
y_train_org = copy.deepcopy(y_train)
y_test_org = copy.deepcopy(y_test)

# Kết quả tập dữ liệu huấn luyện và kiểm tra
x_train.shape, y_train.shape, x_test.shape, y_test.shape


((5469, 40), (5469,), (31610, 40), (31610,))

- Tập huấn luyện (x_train) gồm 5469 mẫu và mỗi mẫu có 40 biến đặc trưng sau khi giảm dữ liệu
- Tập kiểm tra (x_test) gồm 31610 mẫu và mỗi mẫu cũng có 40 biến đặc trưng